## 1.1

Q: Determine if a string has all unique characters.

In [3]:
function all_unique(s) {
    return new Set(s).size == s.length;
}

In [4]:
assert = require('assert');
assert.equal(all_unique(''), true);
assert.equal(all_unique('a'), true);
assert.equal(all_unique('aa'), false);

## 1.2

Q: Given two strings determine if one is a permutation of the other.

The non-optimized solution is simple: compared the sorted strings. The sort will be $O(n\log{n})$, and the comparison will be $O(n)$, so the complete solution will be $O(n^2\log{n})$.

However, this is not the best possible runtime. The best possible runtime is $O(n^2)$ (we would need to interate through each string at least once, at least until a failure condition is met). Here is an optimized psuedocode. This algorithm is indeed $O(n^2)$.

```
function is_permutation(str1, str2):
    # Base cases.
    if |str1| != |str2| return False
    elif |str1| == 0 and |str2| == 0 return False

    m1 = {}  # hash map
    m1_keys = Set()

    # Build the string one character count.
    for char of str1:
        if char in m1_keys:
            m1[char] += 1
        else:
            m1_keys += {char}
            m1[char] = 1

    m2 = {}
    m2_keys = Set()
    # Build the string two character count.
    for char of str2:
        # If the string does not contain it.
        if char not in m1_keys:
            return False

        if char not in m2_keys:
            m2_keys += {char}
            m2[char] = 1
        else:
            m2[char] += 1
            if m2[char] > m1[char]:
                return false

    return true
```

Now tests and implementation.

In [10]:
assert = require('assert');
assert.equal(is_permutation('', ''), true);
assert.equal(is_permutation('a', 'aaa'), false);
assert.equal(is_permutation('abc', 'abc'), true);
assert.equal(is_permutation('abc', 'cba'), true);

In [9]:
function is_permutation(str1, str2) {
    if (str1.length !== str2.length)  return false;
    else if (str1.length === 0 & str2.length == 0)  return true;
    
    let str1_map = {};
    let str1_keys = new Set();
    for (let char of str1) {
        str1_keys.has(char) ? str1_map[char] += 1 : str1_map[char] = 0;
        str1_keys.add(char);        
    }
    
    let str2_map = {};
    let str2_keys = new Set();
    for (let char of str2) {
        if (!str1_keys.has(char))  return false;
        
        str2_keys.has(char) ? str2_map[char] += 1 : str2_map[char] = 0;
        str2_keys.add(char);
        
        if (str1_map[char] > str2_map[char])  return false;        
    }
    
    return true;
}

## 1.3

Q: Replace all spaces in a string with `%20`, in the manner appropriate for URL encoding (e.g. trim ending whitespace).

Again a brute force solution comes immediately:

In [34]:
function whiteparse(s) { return s.trimEnd().replace(/ /g, "%20"); }

In [37]:
assert = require('assert');
assert.equal(whiteparse(''), '');
assert.equal(whiteparse(' '), '');
assert.equal(whiteparse('a b c'), 'a%20b%20c');

Though you have to know some stdlib knick-knacks to implement this. TIL: `trim`, `trimEnd`, and `trimStart`; the equivalent of `strip`, `lstrip`, and `rstrip` in Python.

This is the readable brute-force solution. `trimEnd` is $O(n)$, and an efficient implementation of `replace` is $O(n)$, so this algorithm is $O(n^2)$.

Here's the same algorithm without built-ins...

```
function whiteparse(s):
    replace_idxs = []
    
    # First, skip the whitespace.
    i = |s| - 1
    while True:
        if s[i] == " ":
            continue
        else:
            break

    # Now find the indices of the characters needing replacement.
    for c in range(i):
        if s[c] == " ":
            replace_idxs.push(c)
            
    # Use this to determine the length of the output character array.
    c_a = Array(i + len(replace_idxs) * 3)
    
    i = 0
    for c in s:
        if s[c] == " ":
            c_a[i: i + 3] = "%20"
            i += 3
        else:
            c_a[i] = c
            i += 1
    
    return c_a.join("");
```

In [43]:
function whiteparse(s) {
    if (s.length === 0) return s;
    
    replace_idxs = [];
    
    i = s.length - 1;
    while (true) {
        if (i === 0) return ""
        else if (s[i] === " ") i -= 1
        else break
    }
    
    replace_idxs = [];
    for (let c of [...Array(i).keys()]) {
        c = parseInt(c);
        if (s.charAt(c) === " ") {
            replace_idxs.push(c);
        }
    }
    
    if (replace_idxs.length == 0) return s.slice(0, i);
    
    let c_a = Array(s.length - replace_idxs.length + replace_idxs.length * 3);
    
    i = 0;
    for (let c of s) {
        if (c === " ") {
            [c_a[i], c_a[i + 1], c_a[i +2]] = ["%", "2", "0"];
            i += 3;
        } else {
            c_a[i] = c;
            i += 1;
        }
    }
    
    return c_a.join("");
}

In [44]:
assert = require('assert');
assert.equal(whiteparse(''), '');
assert.equal(whiteparse(' '), '');
assert.equal(whiteparse('a b c'), 'a%20b%20c');

Comments on this problem:

* The first solution requires deep knowledge of the JavaScript stdlib. It's what you would do in production, but probably not impressive in an interview.
* There's a probably $O(n^2)$ operation which is $O(n)$ in amortized time. Instead of iterating over the strings a second time, use the known character-to-replace positions to chunk substrings. I had a hard time getting this solution to work right away. In retrospect, it's just an optimization on this brute-force solution, and I shouldn't have gone for it right away.
* The book uses this solution.
* Character arrays are useful!

## 1.4